In [3]:
from sqlalchemy import create_engine, MetaData, Table, Column
from sqlalchemy import DateTime, Float
import pandas as pd

# --------------------------------------------------------
# 1. CONNECTION TO AIVEN MYSQL
# --------------------------------------------------------
host = "anly-615-project-anlyproject.g.aivencloud.com"
port = 23263
user = "avnadmin"
password = "AVNS_uZtAlXsQZVgdnkwXesP"
database = "defaultdb"

# SSL argument is required for Aiven
connect_args = {"ssl": {"ssl_mode": "REQUIRED"}}

engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}",
    connect_args=connect_args,
    echo=False
)

metadata = MetaData()

# --------------------------------------------------------
# 2. DEFINE TABLE SCHEMA
# --------------------------------------------------------
# We rename columns to be SQL-friendly (no spaces, no starting numbers)
ytm_table = Table(
    "ytm_cleaned",
    metadata,
    Column("date", DateTime, primary_key=True),
    Column("yield_1mo", Float),
    Column("yield_5yr", Float),
    Column("yield_spread", Float)
)

# --------------------------------------------------------
# 3. CREATE TABLE IN MYSQL
# --------------------------------------------------------
metadata.create_all(engine)
print("Table 'ytm_cleaned' created (if not already present).")

# --------------------------------------------------------
# 4. LOAD AND CLEAN DATA
# --------------------------------------------------------
csv_path = r"C:\Users\nurta\Downloads\YTM.csv"
df = pd.read_csv(csv_path)

# Remove all columns except Date, 1 Mo, and 5 Yr
columns_to_keep = ["Date", "1 Mo", "5 Yr"]
df = df[columns_to_keep].copy()

# Convert numeric columns to float, coercing errors
df['1 Mo'] = pd.to_numeric(df['1 Mo'], errors='coerce')
df['5 Yr'] = pd.to_numeric(df['5 Yr'], errors='coerce')

# Drop rows with NaNs (crucial before math operations)
df = df.dropna()

# Create the new difference column: 5 Yr - 1 Mo
df['Spread_5Yr_1Mo'] = df['5 Yr'] - df['1 Mo']

# Ensure the new column is float
df['Spread_5Yr_1Mo'] = df['Spread_5Yr_1Mo'].astype(float)

# Date Conversion
# We convert to datetime objects for SQL compatibility (matching SP500 logic)
# If the database requires strictly YYYY-MM-DD strings, we can convert back, 
# but SQLAlchemy handles datetime objects best.
df['Date'] = pd.to_datetime(df['Date'])

# Rename columns to match the SQL Table Schema defined above
df = df.rename(columns={
    "Date": "date",
    "1 Mo": "yield_1mo",
    "5 Yr": "yield_5yr",
    "Spread_5Yr_1Mo": "yield_spread"
})

# Remove duplicates based on date to prevent Primary Key errors
df = df.drop_duplicates(subset="date")

print(f"Data cleaned. Rows to upload: {len(df)}")
print(df.head())

# --------------------------------------------------------
# 5. INSERT INTO MYSQL
# --------------------------------------------------------
df.to_sql(
    "ytm_cleaned",
    con=engine,
    if_exists="append", # Use 'replace' to overwrite, 'append' to add
    index=False
)

print("YTM data uploaded successfully!")

# --------------------------------------------------------
# 6. VALIDATION
# --------------------------------------------------------
print("\n--- VALIDATION: Check uploaded table ---")
sample_ytm = pd.read_sql("SELECT * FROM ytm_cleaned ORDER BY date DESC LIMIT 5;", engine)
print(sample_ytm)

print("\nAll Done!")

Table 'ytm_cleaned' created (if not already present).


C:\Users\nurta\AppData\Local\Temp\ipykernel_18588\4131595626.py:71: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


Data cleaned. Rows to upload: 5857
        date  yield_1mo  yield_5yr  yield_spread
0 2024-12-31       4.40       4.38         -0.02
1 2024-12-30       4.43       4.37         -0.06
2 2024-12-27       4.44       4.45          0.01
3 2024-12-26       4.45       4.42         -0.03
4 2024-12-24       4.44       4.43         -0.01
YTM data uploaded successfully!

--- VALIDATION: Check uploaded table ---
        date  yield_1mo  yield_5yr  yield_spread
0 2024-12-31       4.40       4.38         -0.02
1 2024-12-30       4.43       4.37         -0.06
2 2024-12-27       4.44       4.45          0.01
3 2024-12-26       4.45       4.42         -0.03
4 2024-12-24       4.44       4.43         -0.01

All Done!
